In [1]:
# Import dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy

In [2]:
# Create a set of random latitude and longitude combinations
lats = np.random.uniform(low=-90.000,high=90.000,size=1500)
lngs = np.random.uniform(low=-180.000,high=180.000,size=1500)
coordinates = list(zip(lats,lngs))
coordinates[:10]


[(-69.64122538835421, 47.74618056229801),
 (31.93298515072688, 93.51976915557788),
 (36.3055746192162, 1.481164733571262),
 (-58.655991414280464, -131.8078923372169),
 (87.24694190456628, -69.76386948610315),
 (26.420979154145385, 117.68838838540631),
 (20.517110008188922, -29.89271225528566),
 (76.67069674121493, -170.19866781875706),
 (68.21948928072291, -107.99160241264649),
 (53.987331567393596, 136.9043154346869)]

In [3]:
# Create a list for holding the cities.
cities = []

# Identify the nearest city for each latitude and longitude combination
for coord in coordinates:
    city = citipy.nearest_city(coord[0],coord[1]).city_name
    
    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

637